In [4]:
import pandas as pd

Read source data

In [7]:
# read in data specifying seperator/delimiter
employeeData = pd.read_csv('Source data.csv', sep=';')

Let's check the shape of our data

In [8]:
employeeData.shape

(64, 6)

let's inspect the first records

In [9]:
employeeData.head()

,Person,Nationality,Rate,Availability,Certification,Language
0,1.0,EU,"$300,00",Q1,2.0,E
1,2.0,Non-EU,"$700,00",Q2,1.0,F
2,3.0,EU,"$830,00",Q2,3.0,G
3,4.0,EU,"$760,00",Q3,3.0,F
4,5.0,EU,"$840,00",Q4,4.0,G


Let's have more edtails about the data and the data types

In [10]:
employeeData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Person         60 non-null     float64
 1   Nationality    60 non-null     object 
 2   Rate           60 non-null     object 
 3   Availability   60 non-null     object 
 4   Certification  60 non-null     float64
 5   Language       60 non-null     object 
dtypes: float64(2), object(4)
memory usage: 3.1+ KB


We have loaded the data but let's save to be sure of the format (Python portability) 

In [11]:
ReformattedData = employeeData

Loaded the data

In [12]:
ReformattedData.to_csv('../ReformattedEmployeeData.csv', index=False)

In the source system, “Nationality” indicates someone’s nationality, and is offered with two values: “EU” and “non-EU”, while in the target system the data concerning “Nationality” is to be offered only as “EU”. In the data transformation phase this value is not affected. 

Just for interest, let's check the different unique values of the rates

In [13]:
employeeData['Rate'].describe()

count            60
unique           20
top        $300,00 
freq              5
Name: Rate, dtype: object

In [14]:
employeeData['Rate'].unique()

array([' $300,00 ', ' $700,00 ', ' $830,00 ', ' $760,00 ', ' $840,00 ',
       ' $1.200,00 ', ' $1.100,00 ', ' $930,00 ', ' $500,00 ',
       ' $535,00 ', ' $600,00 ', ' $725,00 ', ' $620,00 ', ' $120,00 ',
       ' $288,00 ', ' $330,00 ', ' $410,00 ', ' $400,00 ', ' $510,00 ',
       ' $520,00 ', nan], dtype=object)

That's a problem - we have 'nan' values (missing data). So data needs to be cleaned. Let's start by assigning the new headings (before clearning). We will replace the souce data headngs with the new ones. But that does not mean that we have dealt with the data transformation yet of course!

In [62]:
NewEmployeeData = employeeData.rename(columns={'Rate':'Salary','Availability':'Period','Certification':'Level','Language':'Communication'})
            

Let's have a look to see if our new headings are OK. Note, we leave 'PartTime' aside for the moment, as this was not part of the source data. 

In [63]:
NewEmployeeData.head()

,Person,Nationality,Salary,Period,Level,Communication
0,1.0,EU,"$300,00",Q1,2.0,E
1,2.0,Non-EU,"$700,00",Q2,1.0,F
2,3.0,EU,"$830,00",Q2,3.0,G
3,4.0,EU,"$760,00",Q3,3.0,F
4,5.0,EU,"$840,00",Q4,4.0,G


That's good. We have the correct target headings now. Let's check the data. 

In [64]:
print (NewEmployeeData)

    Person Nationality     Salary Period  Level Communication
0      1.0          EU   $300,00     Q1     2.0             E
1      2.0      Non-EU   $700,00     Q2     1.0             F
2      3.0          EU   $830,00     Q2     3.0             G
3      4.0          EU   $760,00     Q3     3.0             F
4      5.0          EU   $840,00     Q4     4.0             G
..     ...         ...        ...    ...    ...           ...
59    60.0          EU   $520,00     Q4     1.0             G
60     NaN         NaN        NaN    NaN    NaN           NaN
61     NaN         NaN        NaN    NaN    NaN           NaN
62     NaN         NaN        NaN    NaN    NaN           NaN
63     NaN         NaN        NaN    NaN    NaN           NaN

[64 rows x 6 columns]


Rows 60 and over do not contain valid data, so they should be filtered out

The code below will return rows without NaN

In [65]:
NewEmployeeData = NewEmployeeData[NewEmployeeData.notna().any(axis=1)]

Let's check the shape of the updated employee data frame

In [66]:
NewEmployeeData.shape

(60, 6)

This looks correct. Indeed looking at the data below, we don't have any more NaN values

In [67]:
print(NewEmployeeData)

    Person Nationality       Salary Period  Level Communication
0      1.0          EU     $300,00     Q1     2.0             E
1      2.0      Non-EU     $700,00     Q2     1.0             F
2      3.0          EU     $830,00     Q2     3.0             G
3      4.0          EU     $760,00     Q3     3.0             F
4      5.0          EU     $840,00     Q4     4.0             G
5      6.0          EU   $1.200,00     Q1     5.0             E
6      7.0      Non-EU   $1.100,00     Q3     5.0             O
7      8.0      Non-EU     $930,00     Q4     5.0             O
8      9.0      Non-EU     $500,00     Q3     4.0             O
9     10.0          EU     $535,00     Q2     1.0             E
10    11.0      Non-EU     $600,00     Q1     2.0             O
11    12.0      Non-EU     $725,00     Q2     3.0             O
12    13.0      Non-EU     $600,00     Q3     3.0             O
13    14.0          EU     $620,00     Q1     4.0             G
14    15.0          EU     $300,00     Q

That's good. We can progress with the clean data now. Let's convert the rate to salary first. 

We will not be able to process salary data unless we covnert them to numbers. Notice below that these are objects (strings)

In [68]:
NewEmployeeData['Salary'].head()

0     $300,00 
1     $700,00 
2     $830,00 
3     $760,00 
4     $840,00 
Name: Salary, dtype: object

Let's define our conversion rate from dollars to euros and then add the conversion column; multiplying vector with scalar values won't do but pair-wise multipliation of vectors will do. 

In [69]:
NewEmployeeData['Conversion']='0.9'

In [70]:
NewEmployeeData.head()

,Person,Nationality,Salary,Period,Level,Communication,Conversion
0,1.0,EU,"$300,00",Q1,2.0,E,0.9
1,2.0,Non-EU,"$700,00",Q2,1.0,F,0.9
2,3.0,EU,"$830,00",Q2,3.0,G,0.9
3,4.0,EU,"$760,00",Q3,3.0,F,0.9
4,5.0,EU,"$840,00",Q4,4.0,G,0.9


In [71]:
NewEmployeeData.dtypes

Person           float64
Nationality       object
Salary            object
Period            object
Level            float64
Communication     object
Conversion        object
dtype: object

Salaries are not numbers (float) yet. let's convert them to numbers. But watch out: we need to take out the $ symbol, so while we convert them to numbers as excercise, we go back to strings to take out the symbol - we need a number without the symbol for applying multiplication. 

In [72]:
NewEmployeeData['Conversion']=NewEmployeeData['Conversion'].astype('float64')

In [73]:
NewEmployeeData['Salary']=NewEmployeeData['Salary'].astype('str')

In [74]:
NewEmployeeData.dtypes

Person           float64
Nationality       object
Salary            object
Period            object
Level            float64
Communication     object
Conversion       float64
dtype: object

In [75]:
NewEmployeeData['Salary'] = NewEmployeeData['Salary'].str.replace('$','')

C:\Users\P301449\AppData\Local\Temp\13/ipykernel_8992/1964421653.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  NewEmployeeData['Salary'] = NewEmployeeData['Salary'].str.replace('$','')


In [76]:
NewEmployeeData.head()

,Person,Nationality,Salary,Period,Level,Communication,Conversion
0,1.0,EU,"300,00",Q1,2.0,E,0.9
1,2.0,Non-EU,"700,00",Q2,1.0,F,0.9
2,3.0,EU,"830,00",Q2,3.0,G,0.9
3,4.0,EU,"760,00",Q3,3.0,F,0.9
4,5.0,EU,"840,00",Q4,4.0,G,0.9


So we got rid of the $ symbol in the Salary column

In [77]:
NewEmployeeData['Salary'] = NewEmployeeData['Salary'].str.replace('.','')

C:\Users\P301449\AppData\Local\Temp\13/ipykernel_8992/3248498346.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  NewEmployeeData['Salary'] = NewEmployeeData['Salary'].str.replace('.','')


Now we get rid of the "." separator in 1.000 => 1000.
Next we bring in the "." for the decimal symbol

In [78]:
NewEmployeeData['Salary'] = NewEmployeeData['Salary'].str.replace(',','.')

In [79]:
NewEmployeeData.head()

,Person,Nationality,Salary,Period,Level,Communication,Conversion
0,1.0,EU,300.00,Q1,2.0,E,0.9
1,2.0,Non-EU,700.00,Q2,1.0,F,0.9
2,3.0,EU,830.00,Q2,3.0,G,0.9
3,4.0,EU,760.00,Q3,3.0,F,0.9
4,5.0,EU,840.00,Q4,4.0,G,0.9


And the Salary number format is now with '.'

In [80]:
NewEmployeeData.dtypes

Person           float64
Nationality       object
Salary            object
Period            object
Level            float64
Communication     object
Conversion       float64
dtype: object

We can 'force' (coerce) the Salary column to be numeric - see below. We are OK to do so as we have finished with string type processing.

In [81]:
NewEmployeeData["Salary"] = pd.to_numeric(NewEmployeeData.Salary, errors='coerce')

In [82]:
NewEmployeeData.dtypes

Person           float64
Nationality       object
Salary           float64
Period            object
Level            float64
Communication     object
Conversion       float64
dtype: object

In [83]:
NewEmployeeData['Salary'] = NewEmployeeData['Salary'] * NewEmployeeData['Conversion']

So now we have been able to do the pair-wise column multiplication for the conversion. Let's have a look at he converted data below. 

In [84]:
NewEmployeeData.head()

,Person,Nationality,Salary,Period,Level,Communication,Conversion
0,1.0,EU,270.0,Q1,2.0,E,0.9
1,2.0,Non-EU,630.0,Q2,1.0,F,0.9
2,3.0,EU,747.0,Q2,3.0,G,0.9
3,4.0,EU,684.0,Q3,3.0,F,0.9
4,5.0,EU,756.0,Q4,4.0,G,0.9


Great! We have now converted the salary from Dollars to Euros. We need one more conversion though. The salary is still in hourly rates though. We need to convert it to weekly rate. Assuming that a week has 40 hours, we will simply multiply by 40. We will use a similar approach as with the $ to euro conversion. 

In [85]:
print(NewEmployeeData)

    Person Nationality  Salary Period  Level Communication  Conversion
0      1.0          EU   270.0    Q1     2.0             E         0.9
1      2.0      Non-EU   630.0    Q2     1.0             F         0.9
2      3.0          EU   747.0    Q2     3.0             G         0.9
3      4.0          EU   684.0    Q3     3.0             F         0.9
4      5.0          EU   756.0    Q4     4.0             G         0.9
5      6.0          EU  1080.0    Q1     5.0             E         0.9
6      7.0      Non-EU   990.0    Q3     5.0             O         0.9
7      8.0      Non-EU   837.0    Q4     5.0             O         0.9
8      9.0      Non-EU   450.0    Q3     4.0             O         0.9
9     10.0          EU   481.5    Q2     1.0             E         0.9
10    11.0      Non-EU   540.0    Q1     2.0             O         0.9
11    12.0      Non-EU   652.5    Q2     3.0             O         0.9
12    13.0      Non-EU   540.0    Q3     3.0             O         0.9
13    

In [86]:
NewEmployeeData['Conversion']='40'
NewEmployeeData['Conversion']=NewEmployeeData['Conversion'].astype('float64')
NewEmployeeData['Salary'] = NewEmployeeData['Salary'] * NewEmployeeData['Conversion']
NewEmployeeData.head(5)

,Person,Nationality,Salary,Period,Level,Communication,Conversion
0,1.0,EU,10800.0,Q1,2.0,E,40.0
1,2.0,Non-EU,25200.0,Q2,1.0,F,40.0
2,3.0,EU,29880.0,Q2,3.0,G,40.0
3,4.0,EU,27360.0,Q3,3.0,F,40.0
4,5.0,EU,30240.0,Q4,4.0,G,40.0


In [87]:
NewEmployeeData['Salary'] = NewEmployeeData['Salary'].astype(float)

Our conversion is complete and we have set the data type to numeric (float)

In [88]:
NewEmployeeData.shape

(60, 7)

In [89]:
NewEmployeeData.head()

,Person,Nationality,Salary,Period,Level,Communication,Conversion
0,1.0,EU,10800.0,Q1,2.0,E,40.0
1,2.0,Non-EU,25200.0,Q2,1.0,F,40.0
2,3.0,EU,29880.0,Q2,3.0,G,40.0
3,4.0,EU,27360.0,Q3,3.0,F,40.0
4,5.0,EU,30240.0,Q4,4.0,G,40.0


So we have completed step 2. Now let's consider step 3: 
"In the source system, “Availability” refers to the period that someone is available is denoted by the value “Q1, Q2, Q3, Q4”, indicating in which quarter of the year that someone is available. In the target system “Period” indicates the interval of weeks that someone is available (e.g. [wk2-wk5] indicates that some is availble in the week 2 up to and including week 5 for work)." Now if someone is available in the Period Q1, this includes wk2-wk5. So we will check for Q1 availability and convert it to start and end week of availability. To do so we will create two more columns: start and end week. We will give them the default values '1' and '52'and we will update them afterwards. 

In [90]:
NewEmployeeData['Start week'] ='1'
NewEmployeeData['End week'] ='52'

We will select all Employees with Q1 availability and will update their start and end weeks. 
We will then do similarly for Employees with Q2, Q3, and Q4 availablity. 

Let's check how many employees are available at each quarter

In [91]:
NewEmployeeData.head()

,Person,Nationality,Salary,Period,Level,Communication,Conversion,Start week,End week
0,1.0,EU,10800.0,Q1,2.0,E,40.0,1,52
1,2.0,Non-EU,25200.0,Q2,1.0,F,40.0,1,52
2,3.0,EU,29880.0,Q2,3.0,G,40.0,1,52
3,4.0,EU,27360.0,Q3,3.0,F,40.0,1,52
4,5.0,EU,30240.0,Q4,4.0,G,40.0,1,52


In [92]:
id_counts = NewEmployeeData['Period'].value_counts()
id_counts.head()

 Q3     16
 Q1     15
 Q2     15
 Q4     14
Name: Period, dtype: int64

Now let's create indices pointing to the employees for each quarter

In [93]:
Q1Employees = NewEmployeeData[NewEmployeeData['Period'].str.contains('Q1')]
Q2Employees = NewEmployeeData[NewEmployeeData['Period'].str.contains('Q2')]
Q3Employees = NewEmployeeData[NewEmployeeData['Period'].str.contains('Q3')]
Q4Employees = NewEmployeeData[NewEmployeeData['Period'].str.contains('Q4')]

In [94]:
Q1Employees.head()

,Person,Nationality,Salary,Period,Level,Communication,Conversion,Start week,End week
0,1.0,EU,10800.0,Q1,2.0,E,40.0,1,52
5,6.0,EU,43200.0,Q1,5.0,E,40.0,1,52
10,11.0,Non-EU,21600.0,Q1,2.0,O,40.0,1,52
13,14.0,EU,22320.0,Q1,4.0,G,40.0,1,52
19,20.0,EU,43200.0,Q1,2.0,E,40.0,1,52


In [95]:
Q2Employees.head()

,Person,Nationality,Salary,Period,Level,Communication,Conversion,Start week,End week
1,2.0,Non-EU,25200.0,Q2,1.0,F,40.0,1,52
2,3.0,EU,29880.0,Q2,3.0,G,40.0,1,52
9,10.0,EU,19260.0,Q2,1.0,E,40.0,1,52
11,12.0,Non-EU,26100.0,Q2,3.0,O,40.0,1,52
18,19.0,Non-EU,30240.0,Q2,3.0,O,40.0,1,52


In [96]:
Q3Employees.head()

,Person,Nationality,Salary,Period,Level,Communication,Conversion,Start week,End week
3,4.0,EU,27360.0,Q3,3.0,F,40.0,1,52
6,7.0,Non-EU,39600.0,Q3,5.0,O,40.0,1,52
8,9.0,Non-EU,18000.0,Q3,4.0,O,40.0,1,52
12,13.0,Non-EU,21600.0,Q3,3.0,O,40.0,1,52
14,15.0,EU,10800.0,Q3,5.0,G,40.0,1,52


In [97]:
Q4Employees.head()

,Person,Nationality,Salary,Period,Level,Communication,Conversion,Start week,End week
4,5.0,EU,30240.0,Q4,4.0,G,40.0,1,52
7,8.0,Non-EU,33480.0,Q4,5.0,O,40.0,1,52
15,16.0,EU,25200.0,Q4,5.0,F,40.0,1,52
16,17.0,EU,29880.0,Q4,5.0,E,40.0,1,52
17,18.0,EU,27360.0,Q4,4.0,E,40.0,1,52


Great. Now for each group of Q1, Q2, Q3, and Q4, we can set the start / end week values. It is more elegant to use the loc methods instead (we see how to do that below, but let's progress for now). 

In [98]:
Q1Employees['Start week'] = 1
Q2Employees['Start week'] = 14
Q3Employees['Start week'] = 27
Q4Employees['Start week'] = 40
Q1Employees['End week'] = 13
Q2Employees['End week'] = 26
Q3Employees['End week'] = 39
Q4Employees['End week'] = 52

C:\Users\P301449\AppData\Local\Temp\13/ipykernel_8992/2583481100.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Q1Employees['Start week'] = 1
C:\Users\P301449\AppData\Local\Temp\13/ipykernel_8992/2583481100.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Q2Employees['Start week'] = 14
C:\Users\P301449\AppData\Local\Temp\13/ipykernel_8992/2583481100.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

In [99]:
Q1Employees.head()

,Person,Nationality,Salary,Period,Level,Communication,Conversion,Start week,End week
0,1.0,EU,10800.0,Q1,2.0,E,40.0,1,13
5,6.0,EU,43200.0,Q1,5.0,E,40.0,1,13
10,11.0,Non-EU,21600.0,Q1,2.0,O,40.0,1,13
13,14.0,EU,22320.0,Q1,4.0,G,40.0,1,13
19,20.0,EU,43200.0,Q1,2.0,E,40.0,1,13


In [100]:
Q2Employees.head()

,Person,Nationality,Salary,Period,Level,Communication,Conversion,Start week,End week
1,2.0,Non-EU,25200.0,Q2,1.0,F,40.0,14,26
2,3.0,EU,29880.0,Q2,3.0,G,40.0,14,26
9,10.0,EU,19260.0,Q2,1.0,E,40.0,14,26
11,12.0,Non-EU,26100.0,Q2,3.0,O,40.0,14,26
18,19.0,Non-EU,30240.0,Q2,3.0,O,40.0,14,26


In [101]:
Q3Employees.head()

,Person,Nationality,Salary,Period,Level,Communication,Conversion,Start week,End week
3,4.0,EU,27360.0,Q3,3.0,F,40.0,27,39
6,7.0,Non-EU,39600.0,Q3,5.0,O,40.0,27,39
8,9.0,Non-EU,18000.0,Q3,4.0,O,40.0,27,39
12,13.0,Non-EU,21600.0,Q3,3.0,O,40.0,27,39
14,15.0,EU,10800.0,Q3,5.0,G,40.0,27,39


In [102]:
Q4Employees.head()

,Person,Nationality,Salary,Period,Level,Communication,Conversion,Start week,End week
4,5.0,EU,30240.0,Q4,4.0,G,40.0,40,52
7,8.0,Non-EU,33480.0,Q4,5.0,O,40.0,40,52
15,16.0,EU,25200.0,Q4,5.0,F,40.0,40,52
16,17.0,EU,29880.0,Q4,5.0,E,40.0,40,52
17,18.0,EU,27360.0,Q4,4.0,E,40.0,40,52


In [103]:
UpdatedEmployees = Q1Employees.append([Q2Employees,Q3Employees,Q4Employees])
UpdatedEmployees.shape

(60, 9)

We don't need the Period and Conversions columns any more. We use the drop method to delete them. 

In [104]:
UpdatedEmployees = UpdatedEmployees.drop(['Period','Conversion'], axis=1)

In [105]:
UpdatedEmployees.head()

,Person,Nationality,Salary,Level,Communication,Start week,End week
0,1.0,EU,10800.0,2.0,E,1,13
5,6.0,EU,43200.0,5.0,E,1,13
10,11.0,Non-EU,21600.0,2.0,O,1,13
13,14.0,EU,22320.0,4.0,G,1,13
19,20.0,EU,43200.0,2.0,E,1,13


In [106]:
UpdatedEmployees.sort_index(inplace=True)

In [107]:
UpdatedEmployees.head()

,Person,Nationality,Salary,Level,Communication,Start week,End week
0,1.0,EU,10800.0,2.0,E,1,13
1,2.0,Non-EU,25200.0,1.0,F,14,26
2,3.0,EU,29880.0,3.0,G,14,26
3,4.0,EU,27360.0,3.0,F,27,39
4,5.0,EU,30240.0,4.0,G,40,52


We have completed step 3

Next we focus on the Certification (now Level) information.
Original 1-5 levels have to be transformed to 1-3 levels. Orignal 1-3 should be transformed to 1. 
Original 4 will now be 2. Original 5 will now be 3. 

In [108]:
UpdatedEmployees.dtypes

Person           float64
Nationality       object
Salary           float64
Level            float64
Communication     object
Start week         int64
End week           int64
dtype: object

We will now use the loc method to locate employees with Level at least 5

In [109]:
L3Employees = UpdatedEmployees.loc[UpdatedEmployees['Level'] >= 5.0]

In [110]:
print(L3Employees)

    Person Nationality   Salary  Level Communication  Start week  End week
5      6.0          EU  43200.0    5.0             E           1        13
6      7.0      Non-EU  39600.0    5.0             O          27        39
7      8.0      Non-EU  33480.0    5.0             O          40        52
14    15.0          EU  10800.0    5.0             G          27        39
15    16.0          EU  25200.0    5.0             F          40        52
16    17.0          EU  29880.0    5.0             E          40        52
23    24.0          EU  19260.0    5.0             F          14        26
25    26.0      Non-EU  43200.0    5.0             O           1        13
27    28.0          EU  33480.0    5.0             E          40        52
28    29.0      Non-EU  18000.0    5.0             O          14        26
29    30.0      Non-EU  19260.0    5.0             O          40        52
38    39.0          EU  30240.0    5.0             E          27        39
39    40.0      Non-EU  4

In [111]:
L3Employees.dtypes

Person           float64
Nationality       object
Salary           float64
Level            float64
Communication     object
Start week         int64
End week           int64
dtype: object

In our transformed data, this will be level 3

In [113]:
L3Employees['Level'] = 3.0

C:\Users\P301449\AppData\Local\Temp\13/ipykernel_8992/939124662.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  L3Employees['Level'] = 3.0


In [114]:
print(L3Employees)

    Person Nationality   Salary  Level Communication  Start week  End week
5      6.0          EU  43200.0    3.0             E           1        13
6      7.0      Non-EU  39600.0    3.0             O          27        39
7      8.0      Non-EU  33480.0    3.0             O          40        52
14    15.0          EU  10800.0    3.0             G          27        39
15    16.0          EU  25200.0    3.0             F          40        52
16    17.0          EU  29880.0    3.0             E          40        52
23    24.0          EU  19260.0    3.0             F          14        26
25    26.0      Non-EU  43200.0    3.0             O           1        13
27    28.0          EU  33480.0    3.0             E          40        52
28    29.0      Non-EU  18000.0    3.0             O          14        26
29    30.0      Non-EU  19260.0    3.0             O          40        52
38    39.0          EU  30240.0    3.0             E          27        39
39    40.0      Non-EU  4

In [116]:
L3Employees.dtypes

Person           float64
Nationality       object
Salary           float64
Level            float64
Communication     object
Start week         int64
End week           int64
dtype: object

We will act smilarly for the other levels

In [117]:
L2Employees = UpdatedEmployees[UpdatedEmployees['Level'] == 4]

In [118]:
L2Employees.dtypes

Person           float64
Nationality       object
Salary           float64
Level            float64
Communication     object
Start week         int64
End week           int64
dtype: object

In [119]:
print(L2Employees)

    Person Nationality   Salary  Level Communication  Start week  End week
4      5.0          EU  30240.0    4.0             G          40        52
8      9.0      Non-EU  18000.0    4.0             O          27        39
13    14.0          EU  22320.0    4.0             G           1        13
17    18.0          EU  27360.0    4.0             E          40        52
24    25.0          EU  30240.0    4.0             G          27        39
30    31.0          EU  21600.0    4.0             G           1        13
37    38.0          EU  27360.0    4.0             F          14        26
41    42.0          EU  22320.0    4.0             E          40        52
47    48.0          EU   4320.0    4.0             G          27        39
50    51.0      Non-EU  18000.0    4.0             G          40        52
54    55.0          EU  14760.0    4.0             F          14        26
57    58.0      Non-EU  14400.0    4.0             O          27        39


In [120]:
L2Employees['Level'] = 2

C:\Users\P301449\AppData\Local\Temp\13/ipykernel_8992/89663372.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  L2Employees['Level'] = 2


In [121]:
print(L2Employees)

    Person Nationality   Salary  Level Communication  Start week  End week
4      5.0          EU  30240.0      2             G          40        52
8      9.0      Non-EU  18000.0      2             O          27        39
13    14.0          EU  22320.0      2             G           1        13
17    18.0          EU  27360.0      2             E          40        52
24    25.0          EU  30240.0      2             G          27        39
30    31.0          EU  21600.0      2             G           1        13
37    38.0          EU  27360.0      2             F          14        26
41    42.0          EU  22320.0      2             E          40        52
47    48.0          EU   4320.0      2             G          27        39
50    51.0      Non-EU  18000.0      2             G          40        52
54    55.0          EU  14760.0      2             F          14        26
57    58.0      Non-EU  14400.0      2             O          27        39


In [122]:
L1Employees = UpdatedEmployees.loc[UpdatedEmployees['Level'] <= 2.0]

In [123]:
L1Employees.loc[:,'Level'] = 1

C:\ProgramData\Anaconda202105\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [124]:
L1Employees.head()

,Person,Nationality,Salary,Level,Communication,Start week,End week
0,1.0,EU,10800.0,1,E,1,13
1,2.0,Non-EU,25200.0,1,F,14,26
9,10.0,EU,19260.0,1,E,14,26
10,11.0,Non-EU,21600.0,1,O,1,13
19,20.0,EU,43200.0,1,E,1,13


Although we could already filter the data, at this stage we are interested in the data integration. Let's sort the employees according to their original order. We first merge the different lists, then sort them. 

In [126]:
QualificationEmployees = L1Employees.append([L2Employees,L3Employees])

In [127]:
QualificationEmployees.sort_index(inplace=True)

In [128]:
QualificationEmployees.head()

,Person,Nationality,Salary,Level,Communication,Start week,End week
0,1.0,EU,10800.0,1.0,E,1,13
1,2.0,Non-EU,25200.0,1.0,F,14,26
4,5.0,EU,30240.0,2.0,G,40,52
5,6.0,EU,43200.0,3.0,E,1,13
6,7.0,Non-EU,39600.0,3.0,O,27,39


In [129]:
print(QualificationEmployees)

    Person Nationality   Salary  Level Communication  Start week  End week
0      1.0          EU  10800.0    1.0             E           1        13
1      2.0      Non-EU  25200.0    1.0             F          14        26
4      5.0          EU  30240.0    2.0             G          40        52
5      6.0          EU  43200.0    3.0             E           1        13
6      7.0      Non-EU  39600.0    3.0             O          27        39
7      8.0      Non-EU  33480.0    3.0             O          40        52
8      9.0      Non-EU  18000.0    2.0             O          27        39
9     10.0          EU  19260.0    1.0             E          14        26
10    11.0      Non-EU  21600.0    1.0             O           1        13
13    14.0          EU  22320.0    2.0             G           1        13
14    15.0          EU  10800.0    3.0             G          27        39
15    16.0          EU  25200.0    3.0             F          40        52
16    17.0          EU  2

In [130]:
QualificationEmployees.head(5)

,Person,Nationality,Salary,Level,Communication,Start week,End week
0,1.0,EU,10800.0,1.0,E,1,13
1,2.0,Non-EU,25200.0,1.0,F,14,26
4,5.0,EU,30240.0,2.0,G,40,52
5,6.0,EU,43200.0,3.0,E,1,13
6,7.0,Non-EU,39600.0,3.0,O,27,39


We now need to add the PartTime column to bring the data to the target format, assigning it a default value 

In [131]:
QualificationEmployees['PartTime']='100'

In [132]:
QualificationEmployees.head(5)

,Person,Nationality,Salary,Level,Communication,Start week,End week,PartTime
0,1.0,EU,10800.0,1.0,E,1,13,100
1,2.0,Non-EU,25200.0,1.0,F,14,26,100
4,5.0,EU,30240.0,2.0,G,40,52,100
5,6.0,EU,43200.0,3.0,E,1,13,100
6,7.0,Non-EU,39600.0,3.0,O,27,39,100


Our data transformation is now complete and we are ready to apply "personnel selection filters"

In [136]:
Selected = QualificationEmployees[(QualificationEmployees['Nationality'] == 'EU') & (QualificationEmployees['Level'] > 1) & QualificationEmployees['Communication'].isin(['E','F']) & (QualificationEmployees['Start week']  < 48) & (QualificationEmployees['End week']  > 50)]

Take a moment to look at the structure of the command. It summarises different filtering methods all in on line. 

In [138]:
Selected.head()

,Person,Nationality,Salary,Level,Communication,Start week,End week,PartTime
15,16.0,EU,25200.0,3.0,F,40,52,100
16,17.0,EU,29880.0,3.0,E,40,52,100
17,18.0,EU,27360.0,2.0,E,40,52,100
27,28.0,EU,33480.0,3.0,E,40,52,100
41,42.0,EU,22320.0,2.0,E,40,52,100


Now both our data transformation and selection are complete!